In [2]:
import click
import datetime
import fnmatch
import logging
import os
import requests
import sys
import telnetlib
import urllib.parse
import time

In [5]:
class Kontrol:
    ipaddr = '192.168.1.1'
    port = 80
    user = 'admin'
    password = ''
    telnet_conn = ''
    urls = dict()
    seqs = dict()
    # throttle (0 is off),rudder (min 1?),elevation (1..127),aileron (1..127),0 + settings (speed mode, inverted flying)
    state = {'time': 1, 'throttle':0, 'rudder':1, 'elevation':1, 'aileron':1, 'settings':0}
    var = 250
    
    def __init__(self, ip, port, user, passw):
        self.ipaddr = ip
        self.port = port
        self.user = user
        self.passw = passw
        self.telnet_conn = telnetlib.Telnet(ip,port)
        time.sleep(2)
        '''
        self.telnet_conn.read_until(b"login: ")
        self.telnet_conn.write(user.encode('ascii') + b"\n")
        self.telnet_conn.read_until(b"Password: ")
        self.telnet_conn.write(passw.encode('ascii') + b"\n")
        '''
        
    def done(self):
        self.telnet_conn.write(b"exit\n")
        self.telnet_conn.close()
        
    def set_state(self, argdict):
        for k,v in argdict.items():
            self.state[k] = v
        print(self.state)
    def get_comm(self):
        checksum = sum([self.state['throttle'],self.state['rudder'],
                        self.state['elevation'],self.state['aileron'],self.state['settings']])
        abytes = bytes([self.var, self.state['throttle'],self.state['rudder'],
                        self.state['elevation'],self.state['aileron'],self.state['settings'], checksum])
        return abytes
    def send_comm(self):
        mess = str(self.get_comm())[2:-1]
        print(mess)
        allm = 'echo -e "'+mess+'" > /dev/ttyAMA1'
        print(allm)
        #self.telnet_conn.write(allm+b"\n")
        print(self.telnet_conn.read_all().decode('ascii'))
        time.sleep(self.state['time'])
    def seq_store(self, seqname, index=0):
        self.seqs[seqname] ={}
        self.seqs[seqname][index] = self.state
        return
    
    def seq_get(self, seqname):
        if seqname in self.seqs:
            return self.seqs[seqname]
        return bytes([[0xfa, 0x40, 0x40, 0x40, 0x00, 0x00]])
    def seq_do(self, seqname):
        seqs = self.seq_get(seqname)
        for seq in seqs:
            self.set_state(seqs[seq])
            self.send_comm()
    def get_state(self):
        return self.state
    def url_for(self, script):
        if script in self.urls:
            return self.urls[script]
        url = 'http://{addr}:{port}/{script}'.format(
            addr=self.ipaddr, port=self.port, script=script)
        self.urls[script] = url
        return url
        
    def download(self, output, path):
        if output is None:
            output = os.path.basename(path)
            
        res = requests.get(self.url_for('get_record.cgi'),
                           params=dict(path=path, json=1, **auth_params()),
                           stream=True)
        
        res.raise_for_status()
        
        LOG.info('writing %s to %s', path, output)
        with open(output, 'w') as fd:
            for chunk in res.iter_content(chunk_size=8192):
                fd.write(chunk)
        
    def auth_params(self):
        return {'user': self.user, 'pwd': self.password}
    def snapshot(self,output):
        if output is None:
            now = datetime.datetime.now()
            output = 'snapshot-{}.jpg'.format(now.isoformat())
        
        res = requests.get(self.url_for('snapshot.cgi'),
                           params=dict(json=1, **self.auth_params()))  # , **auth_params()
        
        res.raise_for_status()
        
        with open(output, 'wb') as fd:
            fd.write(res.content)
    
    def get_params(self): # , output, patterns
        res = requests.get(self.url_for('get_params.cgi'),
                           params=dict(json=1, **self.auth_params()))
        
        res.raise_for_status()
        print(res.json())
        
    def get_status(self): # , output, patterns
        res = requests.get(self.url_for('get_status.cgi'),
                           params=dict(json=1, **self.auth_params()))
        res.raise_for_status()
        print(res.json())
        
    def get_properties(self): # , output, patterns
        res = requests.get(self.url_for('get_properties.cgi'),
                           params=dict(json=1, **self.auth_params()))
        res.raise_for_status()
        print(res.json())
        
    def streamurl(self):
        url = self.url_for('av.asf')
        click.echo('{}?{}'.format(
            url, urllib.parse.urlencode(self.auth_params())))

    
    
    


In [8]:
myKontrol = Kontrol('192.168.1.1', 80, 'admin', '')
#myKontrol.url_for('get_record.cgi')
myKontrol.snapshot(None)
#myKontrol.streamurl()
myKontrol.set_state({'throttle':2, 'time':2})
myKontrol.send_comm()
myKontrol.seq_store('seqName')
myKontrol.seq_get('seqName')
myKontrol.seq_do('seqName')

{'time': 2, 'throttle': 2, 'rudder': 1, 'elevation': 1, 'aileron': 1, 'settings': 0}
\xfa\x02\x01\x01\x01\x00\x05
echo -e "\xfa\x02\x01\x01\x01\x00\x05" > /dev/ttyAMA1


ConnectionResetError: [Errno 54] Connection reset by peer

In [5]:
hex(250)

'0xfa'